In [90]:
import pandas as pd

sectoral_tfp_df_file_path = '../raw_data/productivity_data/major-industry-total-factor-productivity-klems.xlsx'
sectoral_tfp_df = pd.read_excel(sectoral_tfp_df_file_path, header=2)

bds_3_df_file_path = '../raw_data/dynamism_data/bds2022_vcn3.csv'
bds_3_df = pd.read_csv(bds_3_df_file_path)

In [91]:
sectoral_tfp_df['NAICS'].unique()

array(['11', '111-112', '113-115', '21', '211', '212', '213', '22', '23',
       'MN', 'DM', 'ND', '321', '327', '331', '332', '333', '334', '335',
       '3361-3363', '3364-3369', '337', '339', '311-312', '313-314',
       '315-316', '322', '323', '324', '325', '326', '42,44-45', '42',
       '44,45', '48-49', '481', '482', '483', '484', '485', '486',
       '487,488,492', '493', '51', '511', '512', '515,517', '518-519',
       '52-53', '52', '521-522', '523', '524', '525', '53', '531',
       '532-533', '54-81', '54-56', '54', '5411', '5415',
       '5412-5414,5416-5419', '55', '56', '561', '562', '61-62', '61',
       '62', '621', '622-623', '624', '71-72', '71', '711-712', '713',
       '72', '721', '722', '81'], dtype=object)

In [92]:
sectoral_tfp_df['NAICS'] = sectoral_tfp_df['NAICS'].replace({
    'MN': '31-33',  
    'DM': '33',     
    'ND': '31'      
})

In [93]:
sectoral_tfp_df['NAICS'].unique()

array(['11', '111-112', '113-115', '21', '211', '212', '213', '22', '23',
       '31-33', '33', '31', '321', '327', '331', '332', '333', '334',
       '335', '3361-3363', '3364-3369', '337', '339', '311-312',
       '313-314', '315-316', '322', '323', '324', '325', '326',
       '42,44-45', '42', '44,45', '48-49', '481', '482', '483', '484',
       '485', '486', '487,488,492', '493', '51', '511', '512', '515,517',
       '518-519', '52-53', '52', '521-522', '523', '524', '525', '53',
       '531', '532-533', '54-81', '54-56', '54', '5411', '5415',
       '5412-5414,5416-5419', '55', '56', '561', '562', '61-62', '61',
       '62', '621', '622-623', '624', '71-72', '71', '711-712', '713',
       '72', '721', '722', '81'], dtype=object)

In [94]:
three_digit_sectoral_df = sectoral_tfp_df[sectoral_tfp_df['NAICS'].str.len() == 3].copy()

print(sorted(three_digit_sectoral_df['NAICS'].unique()))

['211', '212', '213', '321', '322', '323', '324', '325', '326', '327', '331', '332', '333', '334', '335', '337', '339', '481', '482', '483', '484', '485', '486', '493', '511', '512', '523', '524', '525', '531', '561', '562', '621', '624', '713', '721', '722']


In [95]:
long_df = three_digit_sectoral_df.melt(
    id_vars=['NAICS', 'Industry', 'Basis', 'Measure', 'Units'], 
    var_name='year', 
    value_name='value' 
)

In [96]:
tfp_long = long_df[long_df['Measure'] == 'Total factor productivity'].copy()
output_long = long_df[long_df['Measure'] == 'Sectoral output'].copy()

In [97]:
output_pivot = output_long.pivot_table(
    values='value', 
    index='year', 
    columns='NAICS', 
    aggfunc='sum',
    fill_value=0
)

weights = (
    output_pivot.apply(lambda row: row / (row.sum() + 1e-10), axis=1)
    .stack()
    .reset_index()
    .rename(columns={0: 'weight'})
)

# Merge weights with TFP data
tfp_with_weights = tfp_long.merge(
    weights,
    on=['year', 'NAICS'],
    how='left'
).fillna({'weight': 0})

C:\Users\Lamb\AppData\Local\Temp\ipykernel_22000\4150187125.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  output_pivot = output_long.pivot_table(


In [98]:
tfp_units = tfp_with_weights['Units'].value_counts()
print("Units used for Total factor productivity:\n", tfp_units)

tfp_pivoted = tfp_with_weights.pivot(
    index=['NAICS', 'Industry', 'Basis', 'year', 'weight'],
    columns='Units',
    values='value'
).reset_index()

tfp_pivoted.columns.name = None

tfp_pivoted = tfp_pivoted.rename(columns={
    'Index (2017=100)': 'tfp_index_2017',
    '% Change from previous year': 'tfp_pct_change'
})

print("\nFirst few rows of pivoted TFP data:")
tfp_pivoted.head()

Units used for Total factor productivity:
 Units
Index (2017=100)               1369
% Change from previous year    1369
Name: count, dtype: int64

First few rows of pivoted TFP data:


,NAICS,Industry,Basis,year,weight,tfp_pct_change,tfp_index_2017
0,211,Oil and gas extraction,All workers,1987,0.023260,N.A.,60.776
1,211,Oil and gas extraction,All workers,1988,0.018785,4.8,63.664
2,211,Oil and gas extraction,All workers,1989,0.019410,-5.2,60.359
3,211,Oil and gas extraction,All workers,1990,0.021014,-2.9,58.611
4,211,Oil and gas extraction,All workers,1991,0.018356,1.3,59.37


In [99]:
bds_3_df.head()

,year,vcnaics3,firms,estabs,emp,denom,estabs_entry,estabs_entry_rate,estabs_exit,estabs_exit_rate,job_creation,job_creation_births,job_creation_continuers,job_creation_rate_births,job_creation_rate,job_destruction,job_destruction_deaths,job_destruction_continuers,job_destruction_rate_deaths,job_destruction_rate,net_job_creation,net_job_creation_rate,reallocation_rate,firmdeath_firms,firmdeath_estabs,firmdeath_emp
0,1978,113,12352,12496,103194,98546,2436,19.766,2093,16.982,28686,10186,18500,10.336,29.109,19341,7836,11505,7.952,19.626,9345,9.483,39.253,1553,1556,5409
1,1978,114,1651,1687,11180,10209,417,24.866,397,23.673,4738,1933,2805,18.935,46.412,2810,1324,1486,12.970,27.526,1928,18.886,55.052,297,297,863
2,1978,115,6360,6836,71421,70043,1158,16.878,1208,17.607,23945,8451,15494,12.065,34.186,21540,8508,13032,12.147,30.753,2405,3.434,61.505,840,862,5775
3,1978,211,5258,7004,120136,117269,905,12.990,831,11.928,29725,9942,19783,8.478,25.348,23873,11834,12039,10.091,20.357,5852,4.990,40.715,391,402,1637
4,1978,212,7357,9888,372695,391561,1472,15.018,1299,13.253,65604,22899,42705,5.848,16.754,102838,29574,73264,7.553,26.264,-37234,-9.509,33.509,736,748,10065


In [46]:
bds_3_df = bds_3_df.rename(columns={'vcnaics3': 'NAICS'})

In [50]:
bds_3_df['year'] = bds_3_df['year'].astype(int)
bds_87_on = bds_3_df[bds_3_df['year'] >= 1987]

In [54]:
print("bds_87_on NAICS unique values:", bds_87_on['NAICS'].unique()[:5])
print("tfp_pivoted NAICS unique values:", tfp_pivoted['NAICS'].unique()[:5])

bds_87_on NAICS unique values: [113 114 115 211 212]
tfp_pivoted NAICS unique values: ['211' '212' '213' '321' '322']


In [56]:
print("bds_87_on columns:", bds_87_on.columns.tolist())
print("tfp_pivoted columns:", tfp_pivoted.columns.tolist())

bds_87_on columns: ['year', 'NAICS', 'firms', 'estabs', 'emp', 'denom', 'estabs_entry', 'estabs_entry_rate', 'estabs_exit', 'estabs_exit_rate', 'job_creation', 'job_creation_births', 'job_creation_continuers', 'job_creation_rate_births', 'job_creation_rate', 'job_destruction', 'job_destruction_deaths', 'job_destruction_continuers', 'job_destruction_rate_deaths', 'job_destruction_rate', 'net_job_creation', 'net_job_creation_rate', 'reallocation_rate', 'firmdeath_firms', 'firmdeath_estabs', 'firmdeath_emp']
tfp_pivoted columns: ['NAICS', 'Industry', 'Basis', 'year', 'weight', 'tfp_pct_change', 'tfp_index_2017']


In [66]:
bds_87_on = bds_87_on.copy()
bds_87_on['NAICS'] = bds_87_on['NAICS'].astype(str)
tfp_pivoted['NAICS'] = tfp_pivoted['NAICS'].astype(str)

In [120]:
tfp_pivoted['year'] = tfp_pivoted['year'].astype(int)

# 6) Now merge
merged = pd.merge(
    bds_87_on,
    tfp_pivoted, 
    on=['NAICS','year'],   
    how='inner'
)

In [122]:
print(merged.dtypes)

year                             int32
NAICS                           object
firms                            int64
estabs                           int64
emp                              int64
denom                            int64
estabs_entry                    object
estabs_entry_rate               object
estabs_exit                     object
estabs_exit_rate                object
job_creation                     int64
job_creation_births             object
job_creation_continuers          int64
job_creation_rate_births        object
job_creation_rate              float64
job_destruction                  int64
job_destruction_deaths          object
job_destruction_continuers       int64
job_destruction_rate_deaths     object
job_destruction_rate           float64
net_job_creation                 int64
net_job_creation_rate          float64
reallocation_rate              float64
firmdeath_firms                 object
firmdeath_estabs                object
firmdeath_emp            

In [124]:
object_columns = [
    'estabs_entry', 'estabs_entry_rate', 'estabs_exit', 'estabs_exit_rate',
    'job_creation_births', 'job_creation_rate_births', 'job_destruction_deaths',
    'job_destruction_rate_deaths', 'firmdeath_estabs',
    'tfp_pct_change', 'tfp_index_2017'
]

for col in object_columns:
    merged[col] = pd.to_numeric(merged[col], errors='coerce')

print(merged.dtypes)

year                             int32
NAICS                           object
firms                            int64
estabs                           int64
emp                              int64
denom                            int64
estabs_entry                     int64
estabs_entry_rate              float64
estabs_exit                      int64
estabs_exit_rate               float64
job_creation                     int64
job_creation_births              int64
job_creation_continuers          int64
job_creation_rate_births       float64
job_creation_rate              float64
job_destruction                  int64
job_destruction_deaths           int64
job_destruction_continuers       int64
job_destruction_rate_deaths    float64
job_destruction_rate           float64
net_job_creation                 int64
net_job_creation_rate          float64
reallocation_rate              float64
firmdeath_firms                 object
firmdeath_estabs                 int64
firmdeath_emp            

In [126]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

merged.describe()

,year,firms,estabs,emp,denom,estabs_entry,estabs_entry_rate,estabs_exit,estabs_exit_rate,job_creation,job_creation_births,job_creation_continuers,job_creation_rate_births,job_creation_rate,job_destruction,job_destruction_deaths,job_destruction_continuers,job_destruction_rate_deaths,job_destruction_rate,net_job_creation,net_job_creation_rate,reallocation_rate,firmdeath_estabs,weight,tfp_pct_change,tfp_index_2017
count,1296.000000,1296.000000,1296.000000,1.296000e+03,1.296000e+03,1296.000000,1296.000000,1296.000000,1296.000000,1.296000e+03,1296.000000,1.296000e+03,1296.000000,1296.000000,1.296000e+03,1296.000000,1.296000e+03,1296.000000,1296.000000,1.296000e+03,1296.000000,1296.000000,1296.000000,1296.000000,1260.000000,1296.000000
mean,2004.500000,54065.418981,67767.675926,1.310584e+06,1.301591e+06,7730.979167,10.270720,6808.048611,9.790101,2.071624e+05,74559.801698,1.326026e+05,4.920853,14.293715,1.891669e+05,64569.100309,1.245978e+05,4.755776,13.845778,1.799557e+04,0.447952,24.190044,4674.445216,0.027562,0.767063,94.250076
std,10.392305,96608.319576,122389.195735,2.087011e+06,2.063622e+06,14257.005672,4.770377,12265.231277,3.631869,3.926893e+05,156894.283644,2.457642e+05,4.708577,6.925070,3.489188e+05,121452.502630,2.302359e+05,3.234441,6.196792,1.096836e+05,6.634671,9.696577,8726.956074,0.026230,5.545411,15.912088
min,1987.000000,152.000000,160.000000,1.512000e+03,2.055000e+03,29.000000,2.600000,38.000000,3.424000,2.400000e+02,111.000000,1.290000e+02,0.436000,3.602000,4.450000e+02,204.000000,1.050000e+02,0.390000,2.744000,-1.588972e+06,-54.209000,5.489000,4.000000,0.002737,-47.800000,16.261000
25%,1995.750000,5613.500000,6916.500000,3.434062e+05,3.434355e+05,592.000000,6.858250,636.750000,7.305500,3.688400e+04,8700.750000,2.720550e+04,2.335750,9.675500,3.882575e+04,11822.250000,2.541500e+04,3.001250,9.884500,-6.559250e+03,-1.667500,17.483750,362.500000,0.010048,-1.500000,88.462750
50%,2004.500000,13942.500000,16213.000000,5.973095e+05,5.987760e+05,1646.000000,9.607500,1684.000000,9.145500,7.111150e+04,19400.000000,5.229000e+04,4.165000,13.095000,7.245000e+04,23196.000000,5.052300e+04,4.124500,12.740500,3.319000e+03,1.002500,22.688000,1189.500000,0.017610,0.500000,97.249000
75%,2013.250000,48112.750000,56022.250000,1.354233e+06,1.352021e+06,5862.250000,12.752500,5054.250000,11.635500,1.888415e+05,63412.500000,1.235045e+05,6.272000,17.370500,1.745168e+05,55113.500000,1.154810e+05,5.775000,16.241250,2.363300e+04,3.090500,28.964500,3492.250000,0.036867,2.600000,101.749500
max,2022.000000,454704.000000,635402.000000,1.233226e+07,1.227560e+07,76884.000000,52.235000,71767.000000,48.831000,2.670136e+06,933155.000000,1.812727e+06,76.891000,87.265000,3.014094e+06,859954.000000,2.229294e+06,59.854000,65.888000,1.327078e+06,72.973000,109.855000,46359.000000,0.161923,45.800000,180.413000


In [128]:
merged['firmdeath_firms'] = pd.to_numeric(merged['firmdeath_firms'], errors='coerce')
merged['firms'] = pd.to_numeric(merged['firms'], errors='coerce')
merged['firms_percent_destroyed'] = (merged['firmdeath_firms'] / merged['firms']) * 100

In [130]:
output_path = '../processed_data/three_digit_NAICS_bds_tfp.csv'
merged.to_csv(output_path, index=False)
print(f"Data saved to {output_path}")

Data saved to ../processed_data/three_digit_NAICS_bds_tfp.csv
